# Math23k Analysis Report

## Data Description
| Field             | Annotation                                          |
| --------          | --------------------------------------------------- |
| id                | Id of the problem |
| original_text	    | Original text of the problem |
| equation          | Solution to the problem |
| segmented_text    | Chinese word segmentation of the problem |


In [14]:
import numpy as np
import pandas as pd
import jieba

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [15]:
path = "../raw_data/math23k/raw/train23k.json"

data = pd.read_json(path, orient='records')

## Record Examples

In [39]:
data.head()

,id,original_text,equation,segmented_text
0,946,甲数除以乙数的商是1.5，如果甲数增加20，则甲数是乙的4倍．原来甲数=．,x=20/(4-1.5)*1.5,甲 数 除以 乙 数 的 商 是 1.5 ， 如果 甲 数 增加 20 ， 则 甲 数 是 ...
1,21227,客车和货车分别从A、B两站同时相向开出，5小时后相遇．相遇后，两车仍按原速度前进，当它们相距...,x=196/(80%+((3)/(3+2))-1),客车 和 货车 分别 从 A 、 B 两站 同时 相向 开出 ， 5 小时 后 相遇 ． 相...
2,16892,图书角有书30本，第一天借出了(1/5)，第二天又还回5本，现在图书角有多少本书？,x=30*(1-(1/5))+5,图书 角 有 书 30 本 ， 第一天 借出 了 (1/5) ， 第 二 天 又 还 回 5...
3,8502,甲、乙两车同时从相距230千米的两地相向而行，3小时后两车还相距35千米．已知甲车每小时行4...,x=(230-35)/3-48,甲 、 乙 两车 同时 从 相距 230 千米 的 两地 相向 而 行 ， 3 小时 后 两...
4,23021,果园里有苹果树300棵，比桔树多20%，桔树有多少棵？,x=300/(1+20%),果园 里 有 苹果树 300 棵 ， 比 桔树 多 20% ， 桔树 有 多少 棵 ？


In [40]:
data.describe()

,id
count,21162.000000
mean,11591.289434
std,6692.866886
min,1.000000
25%,5801.250000
50%,11579.500000
75%,17383.750000
max,23162.000000


## The number of problems

In [41]:
len(data['id'].unique())

21162

## Part of missing values for every column

In [42]:
data.isnull().sum() / len(data)

id                0.0
original_text     0.0
equation          0.0
segmented_text    0.0
dtype: float64

In [18]:
def cut_word(text):
    return jieba.lcut(text)
data['content']=data['original_text'].apply(cut_word)
data.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\THY\AppData\Local\Temp\jieba.cache
Loading model cost 0.868 seconds.
Prefix dict has been built succesfully.


,id,original_text,equation,segmented_text,content
0,946,甲数除以乙数的商是1.5，如果甲数增加20，则甲数是乙的4倍．原来甲数=．,x=20/(4-1.5)*1.5,甲 数 除以 乙 数 的 商 是 1.5 ， 如果 甲 数 增加 20 ， 则 甲 数 是 ...,"[甲数, 除以, 乙数, 的, 商是, 1.5, ，, 如果, 甲数, 增加, 20, ，,..."
1,21227,客车和货车分别从A、B两站同时相向开出，5小时后相遇．相遇后，两车仍按原速度前进，当它们相距...,x=196/(80%+((3)/(3+2))-1),客车 和 货车 分别 从 A 、 B 两站 同时 相向 开出 ， 5 小时 后 相遇 ． 相...,"[客车, 和, 货车, 分别, 从, A, 、, B, 两站, 同时, 相向, 开出, ，,..."
2,16892,图书角有书30本，第一天借出了(1/5)，第二天又还回5本，现在图书角有多少本书？,x=30*(1-(1/5))+5,图书 角 有 书 30 本 ， 第一天 借出 了 (1/5) ， 第 二 天 又 还 回 5...,"[图书, 角有, 书, 30, 本, ，, 第一天, 借出, 了, (, 1, /, 5, ..."
3,8502,甲、乙两车同时从相距230千米的两地相向而行，3小时后两车还相距35千米．已知甲车每小时行4...,x=(230-35)/3-48,甲 、 乙 两车 同时 从 相距 230 千米 的 两地 相向 而 行 ， 3 小时 后 两...,"[甲, 、, 乙, 两车, 同时, 从, 相距, 230, 千米, 的, 两地, 相向, 而..."
4,23021,果园里有苹果树300棵，比桔树多20%，桔树有多少棵？,x=300/(1+20%),果园 里 有 苹果树 300 棵 ， 比 桔树 多 20% ， 桔树 有 多少 棵 ？,"[果园, 里, 有, 苹果树, 300, 棵, ，, 比, 桔树, 多, 20%, ，, 桔..."


## delete stopword

In [19]:

def get_stopword():
    s = set()
    with open("../raw_data/stopword/stopword.txt","r",encoding="UTF-8") as f:
        for line in f:
            s.add(line.strip())
    return s

def delete_stopword(words):
    return [w for w in words if w not in stopword]

stopword=get_stopword()
data['content']=data['content'].apply(delete_stopword)
data.head()

,id,original_text,equation,segmented_text,content
0,946,甲数除以乙数的商是1.5，如果甲数增加20，则甲数是乙的4倍．原来甲数=．,x=20/(4-1.5)*1.5,甲 数 除以 乙 数 的 商 是 1.5 ， 如果 甲 数 增加 20 ， 则 甲 数 是 ...,"[甲数, 除以, 乙数, 商是, 1.5, 甲数, 增加, 20, 甲数, 乙, 倍, 甲数]"
1,21227,客车和货车分别从A、B两站同时相向开出，5小时后相遇．相遇后，两车仍按原速度前进，当它们相距...,x=196/(80%+((3)/(3+2))-1),客车 和 货车 分别 从 A 、 B 两站 同时 相向 开出 ， 5 小时 后 相遇 ． 相...,"[客车, 货车, A, B, 两站, 相向, 开出, 小时, 相遇, 相遇, 两车, 按原,..."
2,16892,图书角有书30本，第一天借出了(1/5)，第二天又还回5本，现在图书角有多少本书？,x=30*(1-(1/5))+5,图书 角 有 书 30 本 ， 第一天 借出 了 (1/5) ， 第 二 天 又 还 回 5...,"[图书, 角有, 书, 30, 第一天, 借出, 第二天, 回, 图书, 角有, 本书]"
3,8502,甲、乙两车同时从相距230千米的两地相向而行，3小时后两车还相距35千米．已知甲车每小时行4...,x=(230-35)/3-48,甲 、 乙 两车 同时 从 相距 230 千米 的 两地 相向 而 行 ， 3 小时 后 两...,"[甲, 乙, 两车, 相距, 230, 千米, 两地, 相向, 而行, 小时, 两车, 相距..."
4,23021,果园里有苹果树300棵，比桔树多20%，桔树有多少棵？,x=300/(1+20%),果园 里 有 苹果树 300 棵 ， 比 桔树 多 20% ， 桔树 有 多少 棵 ？,"[果园, 里, 苹果树, 300, 棵, 桔树, 20%, 桔树, 棵]"


## Count of words of problems

In [40]:
def getsize(ser):
    return len(ser)

data['word_count']=data['content'].apply(getsize)
data.head()

,id,original_text,equation,segmented_text,content,word_count
0,946,甲数除以乙数的商是1.5，如果甲数增加20，则甲数是乙的4倍．原来甲数=．,x=20/(4-1.5)*1.5,甲 数 除以 乙 数 的 商 是 1.5 ， 如果 甲 数 增加 20 ， 则 甲 数 是 ...,"[甲数, 除以, 乙数, 商是, 1.5, 甲数, 增加, 20, 甲数, 乙, 倍, 甲数]",12
1,21227,客车和货车分别从A、B两站同时相向开出，5小时后相遇．相遇后，两车仍按原速度前进，当它们相距...,x=196/(80%+((3)/(3+2))-1),客车 和 货车 分别 从 A 、 B 两站 同时 相向 开出 ， 5 小时 后 相遇 ． 相...,"[客车, 货车, A, B, 两站, 相向, 开出, 小时, 相遇, 相遇, 两车, 按原,...",33
2,16892,图书角有书30本，第一天借出了(1/5)，第二天又还回5本，现在图书角有多少本书？,x=30*(1-(1/5))+5,图书 角 有 书 30 本 ， 第一天 借出 了 (1/5) ， 第 二 天 又 还 回 5...,"[图书, 角有, 书, 30, 第一天, 借出, 第二天, 回, 图书, 角有, 本书]",11
3,8502,甲、乙两车同时从相距230千米的两地相向而行，3小时后两车还相距35千米．已知甲车每小时行4...,x=(230-35)/3-48,甲 、 乙 两车 同时 从 相距 230 千米 的 两地 相向 而 行 ， 3 小时 后 两...,"[甲, 乙, 两车, 相距, 230, 千米, 两地, 相向, 而行, 小时, 两车, 相距...",24
4,23021,果园里有苹果树300棵，比桔树多20%，桔树有多少棵？,x=300/(1+20%),果园 里 有 苹果树 300 棵 ， 比 桔树 多 20% ， 桔树 有 多少 棵 ？,"[果园, 里, 苹果树, 300, 棵, 桔树, 20%, 桔树, 棵]",9
